In [ ]:
import pandas as pd
import re 

from prettytable import PrettyTable

In [23]:
pip install pandas openpyxl

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------- ----------------- 143.4/250.9 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Lectura del CSV y guardado en una variable
def READ_CSV(x):
    return pd.read_csv(x, delimiter=",", encoding='latin-1')

FILE = READ_CSV('snaks_bbva_2.csv')

# Guardo los emails en una sola colimna de un DF
bbva = pd.DataFrame(FILE)

In [11]:
# Lectura del CSV y guardado en una variable
def READ_CSV(x):
    return pd.read_csv(x, delimiter=",", encoding='latin-1')

FILE = READ_CSV('snaks_proveedor_2.csv')

# Guardo los emails en una sola colimna de un DF
proveedor = pd.DataFrame(FILE)

In [12]:
def eliminar_caracteres_especiales(texto):
    if isinstance(texto, str):  # Verifica si es una cadena de texto
        return re.sub(r'[^A-Za-z0-9\s]+', '', texto)
    else:
        return texto  # Retorna el valor tal cual si no es una cadena
    
def eliminar_despues_de_guion(texto):
    if isinstance(texto, str):  # Verifica si es una cadena de texto
        return re.sub(r'-.*', '', texto).strip()
    else:
        return texto  # Retorna el valor tal cual si no es una cadena    
    
# Función para eliminar "NRO." del texto
def eliminar_nro(texto):
    if isinstance(texto, str):  # Verifica si es una cadena de texto
        return re.sub(r'NRO\.', '', texto)
    else:
        return texto  # Retorna el valor tal cual si no es una cadena    

In [13]:
# Transformaciones sobre la base del Proveedor

proveedor['direccion'] =  proveedor['Calle'] + ' ' + proveedor['Altura'] 

proveedor['direccion'] =  proveedor['direccion'].apply(eliminar_caracteres_especiales)

proveedor['direccion'] =  proveedor['direccion'].str.upper()

In [14]:
# Transformaciones sobre la base de BBVA

bbva['CALLE_EDIFICIO'] = bbva['CALLE_EDIFICIO'].apply(eliminar_despues_de_guion)

bbva['CALLE_EDIFICIO'] =  bbva['CALLE_EDIFICIO'].apply(eliminar_nro)

bbva['CALLE_EDIFICIO'] = bbva['CALLE_EDIFICIO'].apply(eliminar_caracteres_especiales)

bbva['CALLE_EDIFICIO'] = bbva['CALLE_EDIFICIO'].str.upper()


# Contamos la cantidad de empleados por Direccion y lo guardamos en un diccionario 
bbva['conteo'] = bbva.groupby('CALLE_EDIFICIO')['CALLE_EDIFICIO'].transform('count')

diccionario_bbva = bbva.set_index('CALLE_EDIFICIO')['conteo'].to_dict()

In [15]:
print(diccionario_bbva)

{'ALEM LEANDRO N AVDA 728': 4, 'AV INDEPENDENCIA 2650': 43, 'AV ASAMBLEA 695': 35, 'AV COLON 301': 6, 'RECONQUISTA 2': 71, 'AV ALBERTI 867': 9, 'AV CORRIENTES 1102': 7, 'AV BARTOLOME MITRE 602': 24, 'SAN MARTIN 622': 30, 'BARTOLOME MITRE 215': 18, 'SAN MARTIN 755': 9, 'LEANDRO N ALEM 76': 14, 'COLON 1650': 25, 'AV ANTARTIDA ARGENTINA 1770': 3, 'AVPUEYRREDON 1001': 8, '12 DE OCTUBRE 3264': 7, 'AV ALVEAR 1528': 10, 'AV SANTA FE 1630': 12, 'PETRONA EYLE 398': 9, 'AV SANTA FE 5000': 9, 'AV FCO LACROZE 24515': 7, 'CALLE 47  641': 13, 'AVDA DEL LIBERTADOR 6220': 14, 'AV CORRIENTES 1401': 9, 'AV ESPAA 1342': 15, 'AV PTE PERON 1226': 15, 'AV MONTES DE OCA 1107': 8, 'ALTE BROWN 965': 11, 'AV LIBERTADOR 323': 12, 'SAN ANTONIO 100': 8, 'AVCORRIENTES 942': 7, 'VIAMONTE 1799': 11, 'AV RIVADAVIA 64837': 27, 'AVCABILDO 3622': 9, 'BARTOLOME MITRE 81': 8, 'AV BELGRANO 698': 7, 'TUCUMAN 699': 9, 'SANTIAGO DEL ESTERO 690': 5, 'BRASIL ZONA PUERTO 1': 3, 'ALSINA 216': 17, 'AV ALVEAR 992': 13, 'MUNIAGURRIA 

In [16]:
# DF unificado

df_unificado = pd.DataFrame()

df_unificado['direccion_proveedor'] = proveedor['direccion'].str.strip()

# Función para verificar si el nombre de la sucusal está en el diccionario
def buscar_en_diccionario(valor_a):
    return diccionario_bbva.get(valor_a, None)  # Devuelve None si valor_a no está en el diccionario

# Aplicamos la función a toda la columna A para verificar si cada valor está en el diccionario
df_unificado['cantidad_empleados'] = df_unificado['direccion_proveedor'].apply(buscar_en_diccionario)

# Unificacion con el DF del Proveedor
proveedor['cantidad_empleados'] = df_unificado['cantidad_empleados'] 

# Ordenar el DataFrame de mayor a menor según la columna 'Edad'
proveedor = proveedor.sort_values(by='cantidad_empleados', ascending=False)

In [17]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# Prety Table Format:

df_new_style = PrettyTable()
df_new_style.field_names = proveedor.columns.tolist()
for row in proveedor.itertuples(index=False):
        df_new_style.add_row(row)

 

In [24]:
# Exportar a Excel
archivo_excel = 'cruce_bbva_proveedor.xlsx'  # Nombre del archivo Excel
proveedor.to_excel(archivo_excel, index=False, engine='openpyxl')  # index=False para evitar agregar una columna de índices

In [ ]:
print(df_new_style)


+----------------------------------+---------------------------------+----------+------------------------------+-------------------------------------+---------------------+--------+-------------------------+-----------------------+--------------------------------------+--------------+-------------------------------------+--------------------+
|       Nombre Sucursal BBVA       |              Calle              |  Altura  |        Observaciones         |              Localidad              |      Provincia      |   CP   |          Nombre         |        Apellido       |                 Mail                 | NOMINA TOTAL |              direccion              | cantidad_empleados |
+----------------------------------+---------------------------------+----------+------------------------------+-------------------------------------+---------------------+--------+-------------------------+-----------------------+--------------------------------------+--------------+-------------------------